# Prophet Solution

## Commons



In [1]:
# Python
import pandas as pd
import numpy as np
from fbprophet import Prophet
import utils
import matplotlib
import matplotlib.pyplot as plt
from fbprophet.diagnostics import cross_validation

%matplotlib inline

def rmsle(y_pred, targ):
    log_vars = np.log(targ + 1) - np.log(y_pred + 1)
    return np.sqrt(np.square(log_vars).mean())

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data_raw = pd.read_csv('./data/air_visit_data_with_weather.csv')
test_raw = pd.read_csv('./data/test_data_with_weather.csv')
data = data_raw[['air_store_id', 'visit_date', 'visitors', 
                 'avg_temperature', 'high_temperature', 'low_temperature', 
                 'precipitation', 'avg_wind_speed']]
test = test_raw[['air_store_id', 'ds', 
                 'avg_temperature', 'high_temperature', 'low_temperature', 
                 'precipitation', 'avg_wind_speed']]

In [3]:
data = data.rename({
    'visit_date': 'ds',
    'visitors': 'y'
}, axis='columns')
data['y'] = np.log(data.y)
stores = data.air_store_id.unique()
hol = pd.read_csv('./data/date_info.csv')
hol = hol.rename(
    
    {
        'calendar_date': 'Date',
    }, axis='columns')
hol.Date = pd.to_datetime(hol.Date)
hol = utils.add_ts_elapsed('holiday_flg', ['dur_'], hol)
calender = hol[['Date', 'holiday_flg', 'dur_holiday_flg']]
prophit_hol = calender[calender.holiday_flg == 1].rename({
    'dur_holiday_flg': 'holiday',
    'Date': 'ds'
},axis="columns").drop('holiday_flg', axis=1)
prophit_hol.holiday = prophit_hol.holiday.astype('str')

## ETL

In [4]:
## note: snowfall in test set is constant, thus is removed
data['precipitation'] = data.precipitation.fillna(0)
# data['deepest_snowfall'] = data.deepest_snowfall.fillna(0)
test['precipitation'] = test.precipitation.fillna(0)
# test['deepest_snowfall'] = test.deepest_snowfall.fillna(0)

default_avgt = np.median(data[~data.avg_temperature.isna()].avg_temperature.values)
data['avg_temperature'] = data.avg_temperature.fillna(default_avgt)
test['avg_temperature'] = test.avg_temperature.fillna(default_avgt)

default_hight = np.median(data[~data.high_temperature.isna()].high_temperature.values)
data['high_temperature'] = data.high_temperature.fillna(default_hight)
test['high_temperature'] = test.high_temperature.fillna(default_hight)

default_lowt = np.median(data[~data.low_temperature.isna()].low_temperature.values)
data['low_temperature'] = data.low_temperature.fillna(default_lowt)
test['low_temperature'] = test.low_temperature.fillna(default_lowt)

default_wsp = np.median(data[~data.avg_wind_speed.isna()].avg_wind_speed.values)
data['avg_wind_speed'] = data.avg_wind_speed.fillna(default_wsp)
test['avg_wind_speed'] = test.avg_wind_speed.fillna(default_wsp)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

## Find best model for each store by cross cv

In [28]:
def optStoreModel(store_id):
    store_df = data[data.air_store_id == store_id].copy()
    dl = len(store_df)
    cv_horizon = str(int(dl / 15)) + ' days'
    m0 = Prophet(holidays=prophit_hol, 
                yearly_seasonality=False,
                daily_seasonality=False,
                weekly_seasonality=True)

    m0.fit(store_df)
    cv0 = cross_validation(m0, horizon=cv_horizon)
    
#     m1 = Prophet(holidays=prophit_hol, 
#                 yearly_seasonality=True,
#                 daily_seasonality=False,
#                 weekly_seasonality=True)
#     m1.fit(store_df)
#     cv1 = cross_validation(m1, horizon=cv_horizon)
    return cv0
    
#     return {
#         'air_store_id': store_id,
#         'data_len': dl,
#         'cv0': cv0,
#         'cv1': cv1
#     }

In [31]:
result

{1:             ds      yhat  yhat_lower  yhat_upper         y     cutoff
 0   2016-04-11  2.693852    2.226641    3.184385  2.302585 2016-04-10
 1   2016-04-12  2.872421    2.346585    3.318439  3.091042 2016-04-10
 2   2016-04-13  3.322848    2.847625    3.782145  3.401197 2016-04-10
 3   2016-04-14  3.310487    2.823817    3.795679  3.178054 2016-04-10
 4   2016-04-15  3.698112    3.188549    4.221325  3.931826 2016-04-10
 5   2016-04-16  3.103659    2.644356    3.598085  3.555348 2016-04-10
 6   2016-04-18  2.722454    2.244330    3.190772  3.044522 2016-04-10
 7   2016-04-19  2.901023    2.403086    3.399269  3.135494 2016-04-10
 8   2016-04-20  3.351449    2.881573    3.826224  3.496508 2016-04-10
 9   2016-04-21  3.339088    2.842742    3.810932  2.397895 2016-04-10
 10  2016-04-22  3.726713    3.255801    4.218726  3.688879 2016-04-10
 11  2016-04-23  3.132260    2.636426    3.643461  3.332205 2016-04-10
 12  2016-04-25  2.751055    2.281031    3.234776  3.367296 2016-04-10
 13

In [20]:
?cross_validation

In [29]:
## optimization
result = {}
from multiprocessing import Pool
concurrency = 3
step_task = 1
stores = data.air_store_id.unique()[:2]
num_tasks = len(stores)
with Pool(concurrency) as pool:
    for i, res in enumerate(pool.imap(optStoreModel, stores), 1):
        result[i] = res
#         if i % step_task == 0:
#             print("progress={} %".format(i/num_tasks*100))

In [34]:
result[1]

,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,2016-04-11,2.693852,2.226641,3.184385,2.302585,2016-04-10
1,2016-04-12,2.872421,2.346585,3.318439,3.091042,2016-04-10
2,2016-04-13,3.322848,2.847625,3.782145,3.401197,2016-04-10
3,2016-04-14,3.310487,2.823817,3.795679,3.178054,2016-04-10
4,2016-04-15,3.698112,3.188549,4.221325,3.931826,2016-04-10
5,2016-04-16,3.103659,2.644356,3.598085,3.555348,2016-04-10
6,2016-04-18,2.722454,2.244330,3.190772,3.044522,2016-04-10
7,2016-04-19,2.901023,2.403086,3.399269,3.135494,2016-04-10
8,2016-04-20,3.351449,2.881573,3.826224,3.496508,2016-04-10
9,2016-04-21,3.339088,2.842742,3.810932,2.397895,2016-04-10


391